In [7]:
import torch
import torch.nn as nn

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import torch.optim as optim
from vgg16 import _Vgg16, Block
from trainer.trainer_kfold import TrainerWithKFoldValidation

In [10]:
BATCH_SIZE = 32
EPOCH = 50

In [11]:
transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                              shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                             shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [12]:
vgg_16_adam = _Vgg16(Block, [2, 2, 3, 3, 3])
optimizer_adam = optim.Adam(vgg_16_adam.parameters(), lr=0.001)

In [13]:
summary(vgg_16_adam, (3, 64, 64), device='cuda')

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Block: 2-1                        [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 64, 64]          1,792
|    |    └─BatchNorm2d: 3-2             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-3                    [-1, 64, 64, 64]          --
|    └─Block: 2-2                        [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 64, 64]          36,928
|    |    └─BatchNorm2d: 3-5             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-6                    [-1, 64, 64, 64]          --
|    └─MaxPool2d: 2-3                    [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 128, 16, 16]         --
|    └─Block: 2-4                        [-1, 128, 32, 32]         --
|    |    └─Conv2d: 3-7                  [-1, 128, 32, 32]         73,856
| 

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Block: 2-1                        [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 64, 64]          1,792
|    |    └─BatchNorm2d: 3-2             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-3                    [-1, 64, 64, 64]          --
|    └─Block: 2-2                        [-1, 64, 64, 64]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 64, 64]          36,928
|    |    └─BatchNorm2d: 3-5             [-1, 64, 64, 64]          128
|    |    └─ReLU: 3-6                    [-1, 64, 64, 64]          --
|    └─MaxPool2d: 2-3                    [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 128, 16, 16]         --
|    └─Block: 2-4                        [-1, 128, 32, 32]         --
|    |    └─Conv2d: 3-7                  [-1, 128, 32, 32]         73,856
| 

In [14]:
trainer = TrainerWithKFoldValidation(vgg_16_adam, optimizer_adam, trainset, testset, save_name="vgg16_adam_batch32_fold5", path=None, epoch=EPOCH, batch=BATCH_SIZE)

In [15]:
trainer.training(width=250)

[epoch=  1, batch_cnt=  250] loss: 2.24277
Train Acc: 17 %
Accuracy: 16 %
[epoch=  1, batch_cnt=  500] loss: 2.02221
Train Acc: 20 %
Accuracy: 20 %
[epoch=  1, batch_cnt=  750] loss: 1.95849
Train Acc: 23 %
Accuracy: 23 %
[epoch=  1, batch_cnt= 1000] loss: 1.90515
Train Acc: 23 %
Accuracy: 22 %
[epoch=  1, batch_cnt= 1250] loss: 1.85601
Train Acc: 26 %
Accuracy: 26 %
[epoch=  2, batch_cnt=  250] loss: 1.79100
Train Acc: 26 %
Accuracy: 26 %
[epoch=  2, batch_cnt=  500] loss: 1.78660
Train Acc: 27 %
Accuracy: 27 %
[epoch=  2, batch_cnt=  750] loss: 1.73980
Train Acc: 29 %
Accuracy: 29 %
[epoch=  2, batch_cnt= 1000] loss: 1.73608
Train Acc: 30 %
Accuracy: 30 %
[epoch=  2, batch_cnt= 1250] loss: 1.68148
Train Acc: 31 %
Accuracy: 32 %
[epoch=  3, batch_cnt=  250] loss: 1.66655
Train Acc: 33 %
Accuracy: 33 %
[epoch=  3, batch_cnt=  500] loss: 1.63592
Train Acc: 36 %
Accuracy: 37 %
[epoch=  3, batch_cnt=  750] loss: 1.60156
Train Acc: 38 %
Accuracy: 38 %
[epoch=  3, batch_cnt= 1000] loss: 1.5

Accuracy: 82 %
[epoch= 23, batch_cnt=  500] loss: 0.15351
Train Acc: 95 %
Accuracy: 82 %
[epoch= 23, batch_cnt=  750] loss: 0.15040
Train Acc: 95 %
Accuracy: 82 %
[epoch= 23, batch_cnt= 1000] loss: 0.15406
Train Acc: 96 %
Accuracy: 82 %
[epoch= 23, batch_cnt= 1250] loss: 0.16066
Train Acc: 95 %
Accuracy: 81 %
[epoch= 24, batch_cnt=  250] loss: 0.12961
Train Acc: 96 %
Accuracy: 82 %
[epoch= 24, batch_cnt=  500] loss: 0.12954
Train Acc: 94 %
Accuracy: 80 %
[epoch= 24, batch_cnt=  750] loss: 0.16138
Train Acc: 96 %
Accuracy: 82 %
[epoch= 24, batch_cnt= 1000] loss: 0.15317
Train Acc: 95 %
Accuracy: 81 %
[epoch= 24, batch_cnt= 1250] loss: 0.14858
Train Acc: 96 %
Accuracy: 82 %
[epoch= 25, batch_cnt=  250] loss: 0.12500
Train Acc: 97 %
Accuracy: 82 %
[epoch= 25, batch_cnt=  500] loss: 0.11013
Train Acc: 96 %
Accuracy: 82 %
[epoch= 25, batch_cnt=  750] loss: 0.13729
Train Acc: 96 %
Accuracy: 82 %
[epoch= 25, batch_cnt= 1000] loss: 0.14577
Train Acc: 97 %
Accuracy: 83 %
[epoch= 25, batch_cnt= 

Train Acc: 98 %
Accuracy: 82 %
[epoch= 45, batch_cnt=  750] loss: 0.08093
Train Acc: 98 %
Accuracy: 82 %
[epoch= 45, batch_cnt= 1000] loss: 0.05200
Train Acc: 99 %
Accuracy: 84 %
[epoch= 45, batch_cnt= 1250] loss: 0.08727
Train Acc: 96 %
Accuracy: 81 %
[epoch= 46, batch_cnt=  250] loss: 0.08162
Train Acc: 98 %
Accuracy: 82 %
[epoch= 46, batch_cnt=  500] loss: 0.06489
Train Acc: 98 %
Accuracy: 83 %
[epoch= 46, batch_cnt=  750] loss: 0.04338
Train Acc: 98 %
Accuracy: 83 %
[epoch= 46, batch_cnt= 1000] loss: 0.05381
Train Acc: 99 %
Accuracy: 84 %
[epoch= 46, batch_cnt= 1250] loss: 0.03878
Train Acc: 99 %
Accuracy: 84 %
[epoch= 47, batch_cnt=  250] loss: 0.03104
Train Acc: 98 %
Accuracy: 83 %
[epoch= 47, batch_cnt=  500] loss: 0.05693
Train Acc: 98 %
Accuracy: 82 %
[epoch= 47, batch_cnt=  750] loss: 0.05324
Train Acc: 98 %
Accuracy: 83 %
[epoch= 47, batch_cnt= 1000] loss: 0.05056
Train Acc: 99 %
Accuracy: 83 %
[epoch= 47, batch_cnt= 1250] loss: 0.06641
Train Acc: 98 %
Accuracy: 82 %
[epoch=

TypeError: Object of type 'ndarray' is not JSON serializable

In [19]:
import numpy as np

epoch_loss = np.array(trainer.epoch_loss).flatten()
epoch_acc = np.array(trainer.epoch_acc).flatten()
epoch_tacc = np.array(trainer.epoch_tacc).flatten()

epoch_loss = list(epoch_loss)
epoch_acc = list(epoch_acc)
epoch_tacc = list(epoch_tacc)

In [22]:
import json
with open('epoch.json', 'w') as fp:
    epoch_info = {
        'acc': epoch_acc,
        'loss': epoch_loss,
        'tacc': epoch_tacc
    }
    json.dump(epoch_info, fp)

In [21]:
checkpoint = {
    'model_stat': trainer.model.state_dict(),
    'optimizer_stat': trainer.optimizer.state_dict(),
    'loss': trainer.criterion.state_dict(),
}

torch.save(checkpoint, 'vgg16_adma_batch32.pth')